**Land value notebook**

In [ ]:
import requests
import json
import pandas as pd
import geopandas as gpd

requestString = 'https://data.lacounty.gov/resource/4bch-gpkr.json?City=LOS ANGELES CA&$limit=1000000'
r = requests.get(requestString)
d = json.loads(r.text)
df_2010 = pd.DataFrame(d)
df_2010

In [ ]:
df_2010.info()

In [ ]:
#keep the desired columns
columns = ['ain','situszip','center_lat','center_lon','rollyear','roll_landvalue','sqftmain','roll_impbaseyear','roll_landbaseyear','propertylocation','usecodedescchar1','effectiveyearbuilt','parcelboundarydescription','roll_landbaseyear','location_1']
df_2010_n = pd.DataFrame(df_2010, columns=columns)
df_2010_n

In [ ]:
gdf_2010 = gpd.GeoDataFrame(
    df_2010_n, geometry=gpd.points_from_xy(df_2010_n.center_lon, df_2010_n.center_lat, 
                                          crs='EPSG:4326'))
gdf_2010.head()

In [ ]:
#import neighborhood boundary data
nb= gpd.read_file('/Users/jinglanlin/Desktop/School/01 UCLA/04 Spring 2022/UP 229 DS/Github/UDS-Project--Full-Team-/Data/LA_Times_Neighborhood_Boundaries (1)-Copy1.geojson')
nb.head()

In [ ]:
#join neighborhood boundary with land value data
gdf_2010_sjoin = gpd.sjoin(gdf_2010,nb.to_crs('EPSG:4326'),how="left", predicate='intersects')
gdf_2010_sjoin.head()

In [ ]:
#keep data with the following neighborhood: Koreatown, Westlake, Silver Lake, Echo Park, Downtown, Pico Union
Westlake = gdf_2010_sjoin.loc[gdf_2010_sjoin['name'] == 'Westlake']
Westlake = Westlake.drop(['index_right'], axis=1)
Westlake

In [ ]:
Koreatown = gdf_2010_sjoin.loc[gdf_2010_sjoin['name'] == 'Koreatown']
Koreatown = Koreatown.drop(['index_right'], axis=1)
Koreatown

In [ ]:
Pico_Union = gdf_2010_sjoin.loc[gdf_2010_sjoin['name'] == 'Pico-Union']
Pico_Union = Pico_Union.drop(['index_right'], axis=1)
Pico_Union

In [ ]:
Downtown = gdf_2010_sjoin.loc[gdf_2010_sjoin['name'] == 'Downtown']
Downtown = Downtown.drop(['index_right'], axis=1)
Downtown

In [ ]:
Silver_Lake = gdf_2010_sjoin.loc[gdf_2010_sjoin['name'] == 'Silver Lake']
Silver_Lake = Silver_Lake.drop(['index_right'], axis=1)
Silver_Lake

In [ ]:
Echo_Park = gdf_2010_sjoin.loc[gdf_2010_sjoin['name'] == 'Echo Park']
Echo_Park = Echo_Park.drop(['index_right'], axis=1)
Echo_Park

In [ ]:
join1 = pd.concat([Westlake, Koreatown], axis=0).reset_index(drop=True)
join1

In [ ]:
join2 = pd.concat([join1,Pico_Union], axis=0).reset_index(drop=True)
join2

In [ ]:
join3 = pd.concat([join2,Downtown], axis=0).reset_index(drop=True)
join3

In [ ]:
join4 = pd.concat([join3,Silver_Lake], axis=0).reset_index(drop=True)
join4

In [ ]:
#join all the neighborhoods 
finaljoin = pd.concat([join4,Echo_Park], axis=0).reset_index(drop=True)
finaljoin 

In [ ]:
#import census tracts file
tract = gpd.read_file('/Users/jinglanlin/Desktop/School/01 UCLA/04 Spring 2022/UP 229 DS/Github/UDS-Project--Full-Team-/Data/Census_Tracts_2020.geojson')
tract.head()

In [ ]:
#join census tract with land value data with neighborhood

In [ ]:
gdf_2010_sjoin = gpd.sjoin(finaljoin,tract.to_crs('EPSG:4326'),how="left", predicate='intersects')
gdf_2010_sjoin =gdf_2010_sjoin.rename(columns={'name': 'neighborhood'})
gdf_2010_sjoin

In [ ]:
#save the file to github

In [ ]:
gdf_2010_sjoin.to_file('landvalue_2010_new.geojson', driver='GeoJSON') 